In [1]:
import pandas as pd
import numpy as np

In [2]:
df_abnormal = pd.read_csv("./full_used_data/abnormal_clips_with_info.csv").drop(columns=["id"])
df_normal = pd.read_csv("./full_used_data/normal_clips_with_info.csv").drop(columns=["id"])
vid_info = pd.read_excel("vid_info_202505.xlsx")
numeric_cols = df_normal.columns[1:61]

df_abnormal = df_abnormal.drop(columns=numeric_cols)
df_normal = df_normal.drop(columns=numeric_cols)

In [3]:
abnormal_clip_counts = df_abnormal['prefix'].value_counts()
df_abnormal["clips"] = df_abnormal["prefix"].map(abnormal_clip_counts)

normal_clip_counts = df_normal['prefix'].value_counts()
df_normal["clips"] = df_normal["prefix"].map(normal_clip_counts)

In [4]:
df_pairs = df_abnormal.drop(columns=["filename", "age"])
df_pairs = df_pairs.groupby("prefix").first().reset_index()
df_pairs = df_pairs.rename(columns={"clips": "abnormal_clips", "prefix": "abnormal_video_key", "month": "abnormal_month", "action_type": "abnormal_action_type"})

In [5]:
df_pairs = df_pairs.merge(vid_info, left_on="abnormal_video_key", right_on="id", how="left")
df_pairs = df_pairs.drop(columns=["id", "folder", "action", "age", "action_type", "age_group", "發展遲緩", "備註"])

In [6]:
df_temp = df_normal.drop(columns=["filename", "age"])
df_temp = df_temp.groupby("prefix").first().reset_index()
df_temp = df_temp.rename(columns={"clips": "normal_clips", "prefix": "normal_video_key", "month": "normal_month", "action_type": "normal_action_type"})

In [7]:
df_temp = df_temp.merge(vid_info, left_on="normal_video_key", right_on="id", how="left")
df_temp = df_temp.drop(columns=["id", "folder", "action", "age", "action_type", "age_group", "發展遲緩", "備註"])

In [8]:
df_temp = df_temp.rename(columns={"gender": "normal_gender"})
df_pairs = df_pairs.rename(columns={"gender": "abnormal_gender"})

In [13]:
# 配對 df_pairs 與 df_temp，限制正常影片只能配對一次
def find_best_match(row, df_temp, used_normal_filenames):
    # 篩選條件：行為類型、性別、月齡相同，且未被使用過
    candidates = df_temp[
        (df_temp['normal_action_type'] == row['abnormal_action_type']) &
        (df_temp['normal_gender'] == row['abnormal_gender']) &
        (df_temp['normal_month'] == row['abnormal_month']) &
        (~df_temp['normal_video_key'].isin(used_normal_filenames))
    ]
    if candidates.empty:
        return None
    # 找出 normal_clips 最接近 abnormal_clips 的 row
    candidates = candidates.copy()  # 避免 SettingWithCopyWarning
    candidates['clip_diff'] = (candidates['normal_clips'] - row['abnormal_clips']).abs()
    best_match = candidates.sort_values('clip_diff').iloc[0]
    return best_match

pairing_results = []
used_normal_filenames = set()  # 儲存已配對的正常影片

for idx, row in df_pairs.iterrows():
    match = find_best_match(row, df_temp, used_normal_filenames)
    if match is not None:
        result = row.to_dict()
        for col in match.index:
            result[f'{col}'] = match[col]
        pairing_results.append(result)
        # 加入已配對的 normal_filename
        used_normal_filenames.add(match['normal_video_key'])

df_pairing = pd.DataFrame(pairing_results)
df_pairing


,abnormal_video_key,abnormal_action_type,abnormal_month,abnormal_clips,abnormal_gender,normal_video_key,normal_action_type,normal_month,normal_clips,normal_gender,clip_diff
0,17_N_1_0.5Y(Mosaic),general,6,60,M,295_C_M_26WK_1,general,6,60,M,0
1,17_N_2_0.5Y(Mosaic),general,6,72,M,55_C_M_0.5Y_3,general,6,72,M,0
2,25_N_0.5Y(Mosaic),general,6,126,F,249_C_F_22.29WK,general,6,132,F,6
3,31_N_1_0.5Y(Mosaic),general,6,36,M,57_C_1_M_0.45y(Mosaic),general,6,36,M,0
4,31_N_2_0.5Y(Mosaic),general,6,30,M,201_C_M_0.49Y_1_v,general,6,30,M,0
5,32_N_0.5Y(Mosaic),general,6,90,M,108_C_M_0.47Y_1(Mosaic),general,6,90,M,0
6,35_N_0.6Y_1(Mosaic),general,8,12,F,56_C_F_0.6Y_3,general,8,12,F,0
7,35_N_0.6Y_2(Mosaic),general,8,12,F,15_C_3_0.6y(Mosaic),general,8,18,F,6
8,35_N_0.6Y_3(Mosaic),general,8,24,F,15_C_2_0.6y(Mosaic),general,8,24,F,0
9,376_N_PWS_M_24.29WKS_1,general,6,24,M,100_C_M_0.48Y_2(Mosaic),general,6,24,M,0


In [ ]:
df_pairing = df_pairing.drop(columns=["clip_diff"])
df_pairing.to_csv("./full_used_data/pairs_action_with_info.csv", index=False, encoding="utf-8-sig")